In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import glob, os

os.chdir('../core')
import stitching_ssid as ssid

os.chdir('../../../../pyRRHDLDS/core')
import ssm_scripts

import pprint

In [ ]:
%matplotlib inline
from timeit import default_timer as timer


p, n, m, T = 20, 3, 2, 150

print('\n Chosen system and simulation sizes: ')
print(' (p, n, m, T) = ', (p, n, m, T))

pars_true, options = ssm_scripts.gen_pars(x_dim=n, y_dim=p, u_dim=m, pars_in=None, obs_scheme=None, 
                                 gen_A='full', lts=np.linspace(0.95, 0.99, n), gen_B='random', gen_Q='identity', 
                                 gen_mu0='zero', gen_d='zero', gen_R='zero', diag_R_flag=True)
pars_true['D'] = np.random.normal(size=(p,m))

# Return function sim_sys() that returns output given inputs
sim_sys = ssid.d_system(pars_true, stype='LTI')

# Fix input signal (given system matrices, will fix the output!)
inputs = np.random.normal(0,1,size=(T,m))

# Compute x, y
data, stateseq = sim_sys(inputs)

##############
# new stuff  #
##############

sub_pops = (np.arange(0,10), np.arange(5,15), np.arange(10,20))
r = len(sub_pops)

k,l = n,n
comp_A_from='observability'
observ = np.zeros((k*p, r*n))
L22 = np.nan * np.empty((k*p, k*sum([sub_pops[i].size for i in range(r)])))
for i in range(r):
    data_i = data[:,sub_pops[i]]
    W_i = ssid.input_output_Hankel_mat(data_i, inputs, k, l, N=T-k-l+1)

    start = timer()
    L = np.linalg.qr(W_i, mode='r').T
    L22_i = L[k*m:,k*m:]
    end = timer()
    
    rho_k = np.hstack( [sub_pops[i]+kr*p for kr in range(r)] )
    idx_n = k*sum([sub_pops[j].size for j in range(i)]) + np.arange(k*sub_pops[i].size,dtype=int)
    L22[np.ix_(rho_k, idx_n)] = L22_i.copy()

plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.imshow(L22, interpolation='none')
plt.colorbar()
plt.subplot(1,2,2)
Z = ssid.soft_impute(L22, n, max_iter=5000)
plt.imshow(Z, interpolation='none')
plt.colorbar()

U,s,V = np.linalg.svd(Z)
sqs = np.sqrt(s)
n = np.sum(np.abs(sqs)>1e-3) # latent dimensionality determined from data!
U, sqS, V = U[:,:n], np.diag(sqs[:n]), V[:n,:]
observ = U.dot(sqS)

if comp_A_from == 'observability':
    A = ssid.comp_A(observ, p, k, comp_A_from)
if comp_A_from == 'reachability':
    A = ssid.comp_A(reach, m, l, comp_A_from)

pars_est = {'A' : A, 
            'B' : None,
            'C' : observ[:p,:n],
            'D' : None}

ssid.rotate_latent_bases_obs(np.arange(p), np.arange(p), pars_true, pars_est, overwrite=True)

plt.figure(figsize=(20,8))
plt.subplot(1,3,1)
plt.imshow(pars_true['C'], interpolation='none')
plt.subplot(1,3,2)
plt.imshow(pars_est['C'], interpolation='none')
plt.subplot(1,3,3)
plt.plot(pars_true['C'], pars_est['C'], '.')
plt.show()            

    

In [ ]:
X_true = np.random.normal(size=(10,3)).dot(np.random.normal(size=(3,10)))
X = X_true.copy()

idx_nan =np.unravel_index(np.random.choice(X.size,X.size//10,False),X.shape)
X[idx_nan] = np.nan

plt.figure(figsize=(20,8))
plt.subplot(1,3,1)
plt.imshow(X_true, interpolation='none')

plt.subplot(1,3,2)
plt.imshow(X, interpolation='none')

Z = ssid.soft_impute(X, 3, max_iter=500)
 
plt.subplot(1,3,3)
plt.imshow(Z, interpolation='none')
plt.show()

np.mean( (X_true[idx_nan] - Z[idx_nan])**2 )


In [ ]:
X_true[idx_nan],Z[idx_nan]

# Generate LTI system

$ y_t = C x_t + D u_t $  

$ x_{t+1} = A x_t + B u_t $

$ y_t \in \mathbb{R}^p, x_t \in \mathbb{R}^n ,u_t \in \mathbb{R}^m, t \in \{0, \ldots, T \}  $ 

In [ ]:
p, n, m, T = 4, 3, 2, 100

print('\n Chosen system and simulation sizes: ')
print(' (p, n, m, T) = ', (p, n, m, T))

pars_true, options = ssm_scripts.gen_pars(x_dim=n, y_dim=p, u_dim=m, pars_in=None, obs_scheme=None, 
                                 gen_A='full', lts=np.linspace(0.95, 0.99, n), gen_B='random', gen_Q='identity', 
                                 gen_mu0='zero', gen_d='zero', gen_R='zero', diag_R_flag=True)
pars_true['D'] = np.random.normal(size=(p,m))

# Return function sim_sys() that returns output given inputs
sim_sys = ssid.d_system(pars_true, stype='LTI')

# Fix input signal (given system matrices, will fix the output!)
inputs = np.random.normal(0,1,size=(T,m))

# Compute x, y
data, stateseq = sim_sys(inputs)

# Visualize    
print('\n Generated input, output and state trajectories:')
plt.figure(figsize=(17,3))
plt.subplot(1,3,3)
plt.plot(data)
plt.title('y')
plt.subplot(1,3,2)
plt.plot(stateseq)
plt.title('x')
plt.subplot(1,3,1)
plt.plot(inputs)
plt.title('u')
plt.show()

# Ho-Kalman's method 

Ho-Kalman reconstructs the system matrices $A,B,C,D$ from the impulse responses $G_t$ of the system.

We will use the true system matrices to simulate the impulse reponses $G_t$, as the system ouput $y_t$ of a system at rest ($x_0 = 0$) upon stimulation with an impulse at $t=0$, i.e. $u_0 = 1, u_t = 0$ for $t>0$.

- ground truth input response functions (using ground-truth $A,B,C,D$), 
- system initially at rest x_0 = 0
- impulse responses of length $k+l \geq 2n$ are sufficient
- latent dimensionality $\hat{n}$ is estimated from the (Hankel matrix of) impulse responses
- $(A,B,C)$ has to be minimal for this to work, i.e. $(A,C)$ being observable, $(B,C)$ being reachable
- there is two ways to get $A$ (reachability or observability matrix), both are implemented


In [ ]:
# set-up 
k, l = n, n # algorithm is guaranteed to work (given assumptions are met) if k,l are at least ground-truth n
G_true = ssid.d_calc_impulse_responses(pars_true)
comp_A_from = 'observability' # can get A either from observability or from reachability matrix

# Do Ho-Kalman
pars_est, obs_est , reach_est , _ , n_est = ssid.Ho_Kalman(G_true,k,l,comp_A_from)
pars_est['mu0'] = np.zeros(n)

# Since there is no noise, we can just check system identification by feeding novel inputs into the true and
# our reconstructed system - the outputs should be identical (note this depends on knowing that x(0)=0)
inputs = np.random.normal(0,1,size=(T,m))
data, stateseq = sim_sys(inputs)
data_est, stateseq_est = ssid.d_sim_system_LTI(pars_est, inputs)
print('')
print('true and estimated latent dim. : (n, n_hat) = ', (n, n_est))

err = ssid.data_reconstruction_MSE(data, data_est)
print('\n We check system identification quality by comparing true and estimated outputs to *novel* inputs \n ')
print('worst-cell reconstruction MSE = ', err.max())
print('average reconstruction MSE = ', err.mean())

# Visualize    
plt.figure(figsize=(17,10))
plt.subplot(2,2,1)
plt.plot(data)
plt.title('y')
plt.subplot(2,2,3)
plt.plot(data_est)
plt.title('y_est')
plt.subplot(2,2,2)
plt.plot(stateseq)
plt.title('x')
plt.subplot(2,2,4)
plt.plot(stateseq_est)
plt.title('x_est')
plt.show()

# naive Ho-Kalman's method on systems not at rest

- imulse input $$ u_t =
\left\{
	\begin{array}{ll}
		1 & \mbox{if } t = 0 \\
		0   & \mbox{if } 0 < t < 2n
	\end{array}
\right.$$
- estimating input responses from output data of system
- assuming $x_0 =0$ even though the system initially is **not** at rest


In [ ]:
k, l = n, n # algorithm is guaranteed to work (given assumptions are met) if k,l are at least n
Gs, X0 = ssid.d_est_input_responses(pars_true,k+l,0)
pars_est, obs_est , reach_est , _ , n_est = ssid.Ho_Kalman(Gs[0:],k,l,comp_A_from)

print('\n Applying Ho-Kalman to improper impulse responses of the ground-truth system')
print('(improper means that x(0)~=0, so that the impulse responses superimpose with CA^{t-1}x(0))')

print('\n The estimated parameters B,D will be wrong, but we can correct using the ground-truth x(0)')

print('\n true and estimated D \n')
print(pars_true['D'])
print('')
print(pars_est['D'])
print('\n true and estimated C*B \n')
print(pars_true['C'].dot(pars_true['B']))
print('')
print(pars_est['C'].dot(pars_est['B']))
print('\n true and estimated C*A*B \n')
print(pars_true['C'].dot(pars_true['A'].dot(pars_true['B'])))
print('')
print(pars_est['C'].dot(pars_est['A']).dot(pars_est['B']))

print('\n true and estimated tilde{D} (correcting with true x(0)) \n ')
print(pars_true['D'] + pars_true['C'].dot(X0.T))
print('')
print(pars_est['D'])

print('\n true and estimated tilde{C}*tilde{B} (correcting with true x(0)) \n')
print(pars_true['C'].dot(pars_true['B']) + pars_true['C'].dot(pars_true['A'].dot(X0.T)))
print('')
print(pars_est['C'].dot(pars_est['B']))




# corrected Ho-Kalman's method on systems not at rest

- extended imulse input $$ u_t =
\left\{
	\begin{array}{ll}
        0 & \mbox{if } -n < t < 0 \\
		1 & \mbox{if } t = 0 \\
		0 & \mbox{if } 0 < t < 2n
	\end{array}
\right.$$
- estimating input responses from output data of system
- using Ho-Kalman ill-assuming $x_0 =0$, then correcting the system matrix estimates


In [ ]:
k, l = n, n # algorithm is guaranteed to work (given assumptions are met) if k,l are at least n
pars_est, sqs, n_est, X0_est = ssid.Ho_Kalman_nonzero_init(pars_true, k, l)

print('\n Applying Ho-Kalman to improper impulse responses of the ground-truth system')
print(' and correcting for this using observability and an extended (zero-padded) input sequence')
print('\n true and estimated D')
print(pars_true['D'])
print('')
print(pars_est['D'])
print('\n true and estimated C*B')
print(pars_true['C'].dot(pars_true['B']))
print('')
print(pars_est['C'].dot(pars_est['B']))
print('\n true and estimated C*A*B')
print(pars_true['C'].dot(pars_true['A'].dot(pars_true['B'])))
print('')
print(pars_est['C'].dot(pars_est['A']).dot(pars_est['B']))


# stitching Ho-Kalman for systems not at rest, stitching impulse responses

- extended imulse input $$ u_t =
\left\{
	\begin{array}{ll}
        0 & \mbox{if } -n < t < 0 \\
		1 & \mbox{if } t = 0 \\
		0 & \mbox{if } 0 < t < 2n
	\end{array}
\right.$$
- estimating input responses $G^i_t$ individually for each subpopulation from output data of system
- using Ho-Kalman on each subpopulation, ill-assuming $x_0 =0$ and then correcting the system matrix estimates $(A^i, B^i, C^i, D^i)$
- stitching the subpopulations by stitching the impulse responses ($G^i_t \rightarrow G_t$) before applying Ho-Kalman once more (now for the whole system)


In [ ]:
k, l = n, n # algorithm is guaranteed to work (given assumptions are met) if k,l are at least n
sub_pops = ((0,1), (1,2), (2,3))
method = 'impulses' # stitching using impulse responses
pars_est, n_est = ssid.stitching_Ho_Kalman(pars_true, sub_pops, k, method=method)

print('\n true and estimated D')
print(pars_true['D'])
print('')
print(pars_est['D'])
print('\n true and estimated C*B')
print(pars_true['C'].dot(pars_true['B']))
print('')
print(pars_est['C'].dot(pars_est['B']))
print('\n true and estimated C*A*B')
print(pars_true['C'].dot(pars_true['A'].dot(pars_true['B'])))
print('')
print(pars_est['C'].dot(pars_est['A']).dot(pars_est['B']))

# stitching Ho-Kalman for systems not at rest, aligning system matrices

- extended imulse input $$ u_t =
\left\{
	\begin{array}{ll}
        0 & \mbox{if } -n < t < 0 \\
		1 & \mbox{if } t = 0 \\
		0 & \mbox{if } 0 < t < 2n
	\end{array}
\right.$$
- estimating input responses $G^i_t$ individually for each subpopulation from output data of system
- using Ho-Kalman on each subpopulation, ill-assuming $x_0 =0$ and then correcting the system matrix estimates $(A^i, B^i, C^i, D^i)$
- stitching the subpopulations by explictily rotating the obtained subpopulation system matrices $(A^i, B^i, C^i, D^i)$ using constraints obtained from observability matrices $\mathcal{O}_k$ and reachability matrices $\mathcal{C}_l$ under pairs of subpopulations $i \neq j$


In [ ]:
k, l = n, n # algorithm is guaranteed to work (given assumptions are met) if k,l are at least n
sub_pops = (np.array((0,1)), np.array((1,2)), np.array((2,3)))

method = 'parameters_reach' # '_reach' to use reachability matrix, '_obs' for using observability matrix

pars_est, n_est = ssid.stitching_Ho_Kalman(pars_true, sub_pops, k, method=method)

print('\n true and estimated D')
print(pars_true['D'])
print('')
print(pars_est['D'])
print('\n true and estimated C*B')
print(pars_true['C'].dot(pars_true['B']))
print('')
print(pars_est['C'].dot(pars_est['B']))
print('\n true and estimated C*A*B')
print(pars_true['C'].dot(pars_true['A'].dot(pars_true['B'])))
print('')
print(pars_est['C'].dot(pars_est['A']).dot(pars_est['B']))
